# Generate initial text files for maizsim simulations

Maizsim has a total of X intput files. The files listed here are only ones that require: <br/>
1. customization for individual site-years, or
2. multiple sets for parametetr sensitivity testing

In [1]:
import os
import pandas as pd
import numpy as np
import glob as glob
import datetime
from funcs import tup_convert

### 0. Setup

#### 0.1Making directories to hold initial files & simulation outputs
Code block only needs to be executed once because after that the folders will be created <br/>
if you try executing this code block after the folders exist it won't execute and will throw an error

In [ ]:
"""
siteyears = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/siteyears_filtered.csv', dtype=str, index_col=0)
for i in np.arange(0,len(siteyears)):
    os.mkdir('/home/disk/eos8/ach315/upscale/inits/custom/' + str(siteyears.iloc[i,0]) + '_' + str(siteyears.iloc[i,1]))
"""

In [2]:
siteyears = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/siteyears_filtered.csv', dtype=str, index_col=0)
siteyears.head()

,site,year,doy,pdate
0,722070,1970,61,1970-03-02
1,722070,1972,61,1972-03-01
2,722070,1977,61,1977-03-02
3,722070,1978,75,1978-03-16
4,722070,1979,61,1979-03-02


#### 0.2 Identifying initial files that need to customized for each site-year due to time & date
Other than weather file, init files that need to be customized include:
1. init.txt
2. time.txt
3. climate.txt
4. management.txt

#### 0.3 Specifying some general input variables

In [3]:
directories = glob.glob('/home/disk/eos8/ach315/upscale/inits/custom_pdate/*')
site_info = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/stations_info_9110.csv', 
                        dtype={'USAF': str}, usecols=[0,1,3,4,8,9,10])
site_info.columns = ['site', 'class', 'station', 'state', 'tzone', 'lat', 'lon']

# setting up tab
tab = '    '

### 1. init.txt

setting up constant parameters:

In [4]:
# management
poprow = 7.5 # plants/meter of row
rowsp = 75 # row spacing (cm)
plant_density = 8 # plant density (plants/m2)
rowang = 0
x_seed = 0
y_seed = 195
cec = 0.65
eomult = 0.5
co2 = -1

# output soil file or not
soil_yes = ('0', tab, '1')
soil_no = ('1', tab, '0')

# adjustable parameters: time
start_date = '03/01/'
#sowing_date = '04/01/'
# this will now be customized
end_date = '12/31/'
timestep = 60

# location
alt = 0

# irrigation 
irrigate = 0

creating .txt file for each site-year:

In [5]:
for i in directories:
    init = open(i + '/init.txt', 'w')
    year = i.split('/')[-1].split('_')[-1]
    site = i.split('/')[-1].split('_')[-2]
    
    # customized parameters: location
    lat = site_info[site_info.site == i.split('/')[-1].split('_')[-2]].lat
    lon = site_info[site_info.site == i.split('/')[-1].split('_')[-2]].lon
    alt = 0 # we actually don't have altitude information, so setting all as sea level

    pdate = siteyears[(siteyears.site == site) & (siteyears.year == year)].iloc[0,3]
    pdate_month = pdate.split('-')[1]
    pdate_day = pdate.split('-')[2]
    
    # customized parameters: timing
    start = "'" + start_date + year + "'"
    sowing = "'" + pdate_month + '/' + pdate_day + '/' + year + "'"
    end = "'" + end_date + year + "'"
    
    # setting up text strings
    str1 = '*** initialization data ***\n'
    str2 = tup_convert(['poprow', tab, 'rowsp', tab, 'plant_density', tab, 
                        'rowang', tab, 'x_seed', tab, 'y_seed', tab, 'CEC', tab, 'eomult'])
    str3 = tup_convert(['%.1f' %poprow, tab, '%.1f' %rowsp, tab, '%.1f' %plant_density, tab,
                        '%.1f' %rowang, tab, '%.1f' %x_seed, tab, '%.1f' %y_seed, tab, 
                        '%.2f' %cec, tab, '%.2f' %eomult])
    str4 = tup_convert(['latitude', tab, 'longitude', tab, 'altitude\n'])
    str5 = tup_convert(['%.2f' %lat, tab, '%.2f' %lon, tab, '%.2f\n' %alt])
    str6 = 'autoirrigate\n'
    str7 = '%.0f\n' %irrigate
    str8 = tup_convert(['begin', tab, 'sowing', tab, 'end', tab, 'timestep (mins)\n'])
    str9 = tup_convert([start, tab, sowing, tab, end, tab, '%.0f\n' %timestep])
    str10 = 'output soils data (g03, g04, g05, and g06 files) 1 if true\n'
    str11 = tup_convert(['no soil files', tab, 'outputsoil files\n'])
    str12 = tup_convert(soil_no)
    
    # compiling all strings
    strings = [str1, str2, str3, str4, str5, str6, str7, str8, str9, str10, str11, str12]
       
    # writing out .txt file and clsoing file
    init.writelines(strings)
    init.close()

### 2. time.txt

setting up constant parameters:

In [6]:
# constant parameters
dt = 0.0001
dt_min = 1e-7
dmul1 = 1.3
dmul2 = 0.3
daily_output = '1    0\n'
hourly_output = '0    1\n'
daily_intput = '1    0\n'
hourly_input = '0    1\n'

# adjustable parameters: time 
# 'start_date' & 'end_date' are already defined when creating init.txt

# adjustable parameters: output format
output_timestep = hourly_output
weadata_timestep = hourly_input

creating .txt file for each site-year:

In [7]:
for i in directories:
    # opening text file
    time = open(i + '/time.txt', 'w')
    
    # customizing year and timing
    year = i.split('/')[-1].split('_')[-1]
    init_time = "'" + start_date + year + "'"
    end_time = "'" + end_date + year + "'\n"

    # setting up text strings
    str1 = '*** synchronizer information ***\n'
    str2 = tup_convert(['initial time', tab, 'dt', tab, 'dtmin', tab, 'dmul1', tab, 'dmul2', tab, 'tfin\n'])
    str3 = tup_convert([init_time, tab, '%.4f' %dt, tab, '%.7f' %dt_min, tab, '%.2f' %dmul1, tab, '%.2f' %dmul2, tab, end_time])
    str4 = 'output variables, 1 if true    Daily    Hourly\n'
    str5 = output_timestep
    str6 = 'weather data, 1 if true    Daily     Hourly\n'
    str7 = weadata_timestep

    # compiling strings
    strings = [str1, str2, str3, str4, str5, str6, str7]

    # writing out .txt file and clsoing file
    time.writelines(strings)
    time.close()

### 3. climate.txt

setting up constant parameters:

In [8]:
# constant parameters
bsolar = 3600
btemp = 1
atemp = 0
erain = 0.1
bwind = 1
bir = 1

# climate data input
climate_input = tup_convert(['0', tab, '0', tab, '0', tab, '0', tab, '0', tab, '1', tab, '1\n'])

# average values for site
# 19118: I'm actually not sure how these values are calculated and how important they are?
winda = 3
irav = 1
conc = 0
co2 = 400

creating .txt file for each site-year:

In [9]:
for i in directories:
    # opening text file
    climate = open(i + '/climate.txt', 'w')
    
    # customizing latitude    
    site = i.split('/')[-1].split('_')[-2]
    lat = site_info[site_info.site == i.split('/')[-1].split('_')[-2]].lat
    
    # setting up text strings
    str1 = '*** standard meteorological data ***\n'
    str2 = 'latitude\n'
    str3 = '%.2f\n' %lat
    str4 = 'daily bulb temp, daily wind, rain intensity, daily conc, furrow, relitive humidity, co2\n'
    str5 = climate_input
    str6 = 'parameters for changing of units: BSOLAR BTEMP ATEMP ERAIN BWIND BIR\n'
    str7 = 'BSOLAR is 1e6/3600 to go from jm-2h-1 to wm-2\n'
    str8 = tup_convert(['%.0f' %bsolar, tab, '%.0f' %btemp, tab, '%.0f' %atemp, tab, '%.0f' %erain, tab, '%.0f' %bwind, tab, '%.0f\n' %bir])
    str9 = 'average values for the site\n'
    str10 = tup_convert(['WINDA', tab, 'IRAV', tab, 'Conc', tab, 'CO2\n'])
    str11 = tup_convert(['%.0f' %winda, tab, '%.0f' %conc])

    # compiling text strings
    strings = [str1, str2, str3, str4, str5, str6, str7, str8, str9, str10, str11]
    
    # writing out .txt file and clsoing file
    climate.writelines(strings)
    climate.close()

### 4. management.txt

setting up constant parameters:

In [10]:
# constant parameters

# adjustable parameters: N application
appl_num = 2 # number of applications
#appl_date_1 = '04/15/' # here are only 2 application dates, but will need to be adjusted if appl_num increases
#appl_date_2 = '05/15/'
appl_mg = 100
appl_depth = 5 # (cm)
residue_C = 0
residue_N = 0

creating .txt file for each site-year:

In [11]:
for i in directories:
    # opening text file
    management = open(i + '/management.txt', 'w')
    
    # customizing N application date
    year = i.split('/')[-1].split('_')[-1]
    site = i.split('/')[-1].split('_')[-2]
    
    # setting up N application dates: 2 weeks & 1.5 months after planting
    pdate = siteyears[(siteyears.site == site) & (siteyears.year == year)].iloc[0,3]
    app1 = pd.to_datetime(pdate).dayofyear + 14
    app2 = pd.to_datetime(pdate).dayofyear + 14 + 30
#    app1 = pd.to_datetime(pdate[0]).dayofyear + 14
#    app2 = pd.to_datetime(pdate[0]).dayofyear + 14 + 30
    appl_date_1 = (datetime.datetime(int(year),1,1) + datetime.timedelta(app1 - 1)).strftime("%Y/%m/%d")
    appl_date_2 = (datetime.datetime(int(year),1,1) + datetime.timedelta(app2 - 1)).strftime("%Y/%m/%d")
    
    # formatting date specifically for maizsim
    date_1 = "'" + appl_date_1 + "'"
    date_2 = "'" + appl_date_2 + "'"

    #    date_1 = "'" + appl_date_1 + year + "'"
    #    date_2 = "'" + appl_date_2 + year + "'"

    # setting up strings
    str1 = '*** script for chemical application module ***\n'
    str2 = 'number of fertilizer applications (max=25)\n'
    str3 = '%.0f\n' %appl_num
    str4 = 'appl_time(i)' + tab + 'appl_mg(i)' + tab + 'appl_depth(cm)' + tab + 'residue_C' + tab + 'residue_N\n'
    # N application unit: kg/ha 
    # total mg N applied to grid (1 kg/ha = 1 mg/m2/width of application) application divided by width of grid in cm is kg/ha
    str5 = tup_convert([date_1, tab, '%.0f' %appl_mg, tab, '%.0f' %appl_depth, tab, '%.0f' %residue_C, tab, '%.0f\n' %residue_N])
    str6 = tup_convert([date_2, tab, '%.0f' %appl_mg, tab, '%.0f' %appl_depth, tab, '%.0f' %residue_C, tab, '%.0f\n' %residue_N])
    
    # compiling strings
    strings = [str1, str2, str3, str4, str5, str6]
    
    # writing out .txt file and clsoing file
    management.writelines(strings)
    management.close()